<a href="https://colab.research.google.com/github/AnviTendulkar/Airline_Fare_Prediction/blob/master/fare_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler ,FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import re
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import StandardScaler, PowerTransformer ,OneHotEncoder
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

In [50]:
pd.set_option("display.max_columns" ,None)
pd.set_option("display.max_rows" ,None)

In [62]:
train_data = pd.read_excel(r'/content/drive/MyDrive/Project/Airline Fare  Prediction/Data_Train.xlsx')

In [63]:
train_data.head(3)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882


In [64]:
train_data.dropna(inplace=True)

In [65]:
X = train_data.loc[:,train_data.columns !='Price']
y =train_data['Price']

In [55]:
x_train ,x_test ,y_train ,y_test = train_test_split(X,y ,test_size=0.20)

In [69]:
print(x_train.head(1))
print(pd.DataFrame(x_test.head(1)))

         Airline Date_of_Journey Source Destination            Route Dep_Time  \
680  Jet Airways      18/05/2019  Delhi      Cochin  DEL → BOM → COK    07:05   

    Arrival_Time Duration Total_Stops Additional_Info  
680        19:00  11h 55m      1 stop         No info  
        Airline Date_of_Journey   Source Destination            Route  \
7105  Air India      21/05/2019  Kolkata    Banglore  CCU → HYD → BLR   

     Dep_Time  Arrival_Time Duration Total_Stops Additional_Info  
7105    19:00  11:05 22 May   16h 5m      1 stop         No info  


In [82]:
# def hour_minute_extractor(Duration):
#     pattern = re.compile(r'(?:(\d+)h)?\s?(?:(\d+)m)?')
#     match = pattern.match(Duration)
#     if match:
#         hours = int(match.group(1)) if match.group(1) else 0
#         minutes = int(match.group(2)) if match.group(2) else 0
#         return pd.Series({'Hour' :hours , 'Minutes':minutes})

def Arrival_Date_calculation(row):
    pattern = re.compile(r'(?:(\d+)h)?\s?(?:(\d+)m)?')
    match = pattern.match(Duration)
    if match:
        hours = int(match.group(1)) if match.group(1) else 0
        minutes = int(match.group(2)) if match.group(2) else 0
        # return pd.Series({'Hour' :hours , 'Minutes':minutes})

    # Concatenate 'Date_of_Journey' and 'Dep_Time'
    date_of_journey = row['Date_of_Journey'] + ' ' + row['Dep_Time']

    # Convert the concatenated string to datetime
    journey_date_time = datetime.strptime(date_of_journey, "%d/%m/%Y %H:%M")

    # Calculate arrival date by adding 'Duration_hour' and 'Duration_minute' to 'Date_of_Journey'
    # arrival_date = journey_date_time + timedelta(hours=row['Duration_hour'], minutes=row['Duration_minute'])
    arrival_date = journey_date_time + timedelta(hours=hours, minutes=minutes)
    return arrival_date

def split_dep_time_in_hours_min(str1):
    lst = str1.split(':')
    return pd.Series({'Dep_Hour': lst[0], 'Dep_Minute': lst[1]})

In [ ]:
from sklearn.base import TransformerMixin
import pandas as pd

class DateColumnExtractor(TransformerMixin):
    def fit(self, X, y=None):
        return self

    # def transform(self, X):
    #     # Assuming X is a DataFrame with a column named 'date_column'
    #     # Replace 'date_column' with the actual name of your string column

    #     # Convert the string column to a pandas datetime object

    #     X[['Duration_hour' , 'Duration_minute']] = X['Duration'].apply(hour_minute_extractor)

    #     print("1")

    #     X['Arrival_Date'] = X.apply(Arrival_Date ,axis=1)
    #     X['Arrival_Year'] = X['Arrival_Date'].dt.year
    #     X['Arrival_Month'] = X['Arrival_Date'].dt.month
    #     X['Arrival_Day'] = X['Arrival_Date'].dt.day
    #     X['Arrival_Hour'] = X['Arrival_Date'].dt.hour
    #     X['Arrival_Minute'] = X['Arrival_Date'].dt.minute


    #     # temp = X['Date_of_Journey'] = pd.to_datetime(X['Date_of_Journey'], errors='coerce')
    #     temp = X['Date_of_Journey'].copy()
    #     temp =  pd.to_datetime(temp)
    #     # Extract year, month, and day into separate columns
    #     X['Dep_Year'] = temp.dt.year
    #     X['Dep_Month'] = temp.dt.month
    #     X['Dep_Day'] = temp.dt.day

    #     X[['Dep_Hour', 'Dep_Minute']] = X['Dep_Time'].apply(split_dep_time_in_hours_min)
    #     return X
    def transform(self, X):
        # Assuming X is a DataFrame with a column named 'date_column'
        # Replace 'date_column' with the actual name of your string column

        # Convert the string column to a pandas datetime object
        temp_df =pd.DataFrame()
        # temp_df[['Duration_hour' , 'Duration_minute']] = X['Duration'].apply(hour_minute_extractor)

        print("1")

        X['Arrival_Date'] = X.apply(Arrival_Date_calculation ,axis=1)
        temp_df['Arrival_Year'] = X['Arrival_Date'].dt.year
        temp_df['Arrival_Month'] = X['Arrival_Date'].dt.month
        temp_df['Arrival_Day'] = X['Arrival_Date'].dt.day
        temp_df['Arrival_Hour'] = X['Arrival_Date'].dt.hour
        temp_df['Arrival_Minute'] = X['Arrival_Date'].dt.minute


        # temp = X['Date_of_Journey'] = pd.to_datetime(X['Date_of_Journey'], errors='coerce')
        temp = X['Date_of_Journey'].copy()
        temp =  pd.to_datetime(temp)
        # Extract year, month, and day into separate columns
        temp_df['Dep_Year'] = temp.dt.year
        temp_df['Dep_Month'] = temp.dt.month
        temp_df['Dep_Day'] = temp.dt.day

        temp_df[['Dep_Hour', 'Dep_Minute']] = X['Dep_Time'].apply(split_dep_time_in_hours_min)
        return temp_df





In [58]:
x_train.head(1)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
680,Jet Airways,18/05/2019,Delhi,Cochin,DEL → BOM → COK,07:05,19:00,11h 55m,1 stop,No info


In [78]:
print(x_train['Airline'].nunique())
print(x_train['Source'].nunique())
print(x_train['Destination'].nunique())
print(x_train['Total_Stops'].nunique())

12
5
6
4


In [ ]:
# # Display the transformed DataFrame

preprocessor = ColumnTransformer([
    ('one-hot',OneHotEncoder(drop = 'first' ,sparse_output =False ,dtype ='int16'),['Airline' ,'Source','Destination' ]),
    ('total_stop',FunctionTransformer(lambda x:x.replace({'1 stops':1,'1 stop':1, 'non-stop':0, '2 stops':2, '3 stops':3, '4 stops':4})),['Total_Stops'])
] ,remainder = 'passthrough')

pipe2 = Pipeline(steps=[
                       ('experimental_trans', DateColumnExtractor()),
                       ('preprocessor', preprocessor)
])
data = pipe2.fit_transform(x_train[['Airline','Date_of_Journey','Source','Destination','Dep_Time','Duration','Total_Stops']])
data = pd.DataFrame(data)
print(data.head(2))

In [ ]:
x_train.head(1)

In [ ]:
x_train.head(1)